# Jane Street Market Prediction - Submission
This is the actual notebook submitted to the competition. First, we load the trained models and use them to implement the `predict()` function.

In [ ]:
import os

import pandas as pd

# load models
model = None

# function to return predictions for the current trade
# in test_df, using historical data in hist_df
def predict(test_df, hist_df):
    return 0

Then we use the Jane Street API to make the predictions. In this setup the iterator `iter_test` yields single samples `test_df` in chronological order. The predict function needs to decide whether to accept or reject the trade at this time-point using only the current sample and the previous samples stored in `hist_df`. Set `pred_df` to 1 to accept the trade or to 0 to reject the trade.

In [ ]:
import janestreet

env = janestreet.make_env()
iter_test = env.iter_test()

hist_df = pd.DataFrame()

for (test_df, pred_df) in iter_test:
    # accept/reject current trade
    pred_df.action = predict(test_df, hist_df)
    env.predict(sample_prediction_df)
    
    # store the new sample in the history data frame
    hist_df.append(test_df)